# Limpieza de Netflix

## Pedir datos a la API

In [ ]:
# IMPORTS
import numpy as np
import pandas as pd
import requests as rq
from src.buscar_peliculas import buscar_peliculas

### Pedimos los datos

In [6]:
dfnetflix_raw = buscar_peliculas("Netflix")

<Response [200]> 12 2909


In [7]:
largo_dataframe = len(dfnetflix_raw[0])
largo_segun_API = dfnetflix_raw[1]
print([largo_dataframe, largo_segun_API])

[2909, 2909]


In [8]:
dfnetflix1 = dfnetflix_raw[0]

In [9]:
dfnetflix1

,id,title,year,imdb_id,tmdb_id,tmdb_type,type
0,1823218,The Woman in Cabin 10,2025,tt7130300,1290879,movie,movie
1,11042350,"My Father, the BTK Killer",2025,tt38466379,1550785,movie,movie
2,1812460,She Walks in Darkness,2025,tt32129665,1275151,movie,movie
3,1996393,Swim to Me,2025,tt34682204,1484640,movie,movie
4,1944218,Everybody Loves Me When I'm Dead,2025,tt35669054,1429750,movie,movie
...,...,...,...,...,...,...,...
2904,1720462,Jurnal Risa by Risa Saraswati,2024,tt28448409,1141132,movie,movie
2905,1857339,Maple Leaf Dreams,2024,tt27197375,1102686,movie,movie
2906,1852883,Ahir Shah: Ends,2024,tt33246651,1332077,movie,movie
2907,1795372,Harta Tahta Raisa,2024,tt31495242,1250870,movie,movie


Guardamos estos datos crudos en un csv

In [18]:
dfnetflix1.to_csv('data/raw_api_netflix.csv', index=False)

## Limpieza
### Columnas
Las columnas nos entregan los siguientes datos:
- `id`: Identificador de la pelicula en la API
- `title`: Nombre de la pelicula
- `year`: Año de estreno de la pelicula
- `imdb_id`: Identificador de la pelicula en IMDB
- `tmdb_id`: Identificador de la pelicula en TMDB
- `tmdb_type`: Tipo del titulo en TMDB
- `type`: Tipo del titulo en la API

### Eliminar Columnas
Como le especificamos a la API que nos entregara solo peliculas, la columna `'type'` y `'tmdb_type'` no deberian entregar informacion importante, ya que todas las filas tienen el mismo tipo

In [10]:
dfnetflix1 = dfnetflix1
dfnetflix1['type'].unique()

array(['movie'], dtype=object)

In [11]:
dfnetflix1['tmdb_type'].unique()

array(['movie', 'tv'], dtype=object)

En la columna `'type'` si tiene un solo tipo, pero al parecer `'tmdb_type'` no, revisemos las filas con tipo distinto

In [21]:
dfnetflix1[dfnetflix1['tmdb_type'] == 'tv']

,id,title,year,imdb_id,tmdb_id,tmdb_type,type
2790,3199048,The Rise of Wagner,2023,tt27480377,228220,tv,movie


Revisando el titulo 'The Rise of Wagner' en la pagina de imdb, este figura como un documental, al no estar separado en capitulos lo dejaremos como tipo `'movie'`. Al ya ser todos del mismo tipo, ambas columnas pueden ser eliminadas

In [12]:
dfnetflix2 = dfnetflix1[['id', 'title', 'year', 'imdb_id', 'tmdb_id']]

### Valores nan

In [13]:
dfnetflix2['id'].isna().unique()

array([False])

In [14]:
dfnetflix2['title'].isna().unique()

array([False])

In [15]:
dfnetflix2['year'].isna().unique()

array([False])

In [16]:
dfnetflix2['imdb_id'].isna().unique()

array([False])

### Duplicados

In [18]:
dfnetflix2.duplicated().sum()

np.int64(1)

In [17]:
dfnetflix2[dfnetflix2.duplicated()]

,id,title,year,imdb_id,tmdb_id
258,3233385,Saturday Night,2024,tt27657135,1120911


In [19]:
dfnetflix4 = dfnetflix2.drop_duplicates(keep='first')

#### imdb_id

In [20]:
dfnetflix4.duplicated('imdb_id').sum()

np.int64(0)

### Valores unicos

#### id
Deberia ser unico para cada pelicula

In [21]:
dfnetflix4['id'].value_counts()

id
1823218     1
11042350    1
1812460     1
1996393     1
1944218     1
           ..
1720462     1
1857339     1
1852883     1
1795372     1
1747428     1
Name: count, Length: 2908, dtype: int64

#### title
Es posible que se repita, ya que hay peliculas con el mismo titulo

In [23]:
dfnetflix4.duplicated('title').sum()

np.int64(39)

revisemos porque se repiten los titulos de las peliculas

In [24]:
dfnetflix4[dfnetflix4.duplicated('title')]

,id,title,year,imdb_id,tmdb_id
271,1382254,The Call,2013,tt1911644,158011
444,1400061,The Karate Kid,1984,tt0087538,1885
795,1467729,White Noise,2005,tt0375210,11804
938,1678479,Leo,2023,tt15654328,949229
963,1629411,Beast,2022,tt11301946,800383
970,1461727,Wanted,2009,tt1084972,22717
1397,1647850,Don,2022,tt10709484,895033
1484,1604636,HIT: The First Case,2020,tt11490170,678391
1528,1571201,Maria,2019,tt7594584,591121
1541,118794,Agneepath,1990,tt0098999,37403


Como podemos ver por la diferencia de años y el 'imdb_id', puede que sean peliculas distintas con el mismo titulo, revisando la pagina de imdb se confirma esto

### Tipos de columnas

In [25]:
dfnetflix4.dtypes

id          int64
title      object
year        int64
imdb_id    object
tmdb_id     int64
dtype: object

#### year

In [26]:
dfnetflix4[dfnetflix4['year'] == 0]

,id,title,year,imdb_id,tmdb_id
2880,11039715,Rockstar: DUKI desde el fin del mundo,0,tt38203421,1546933
2890,11046064,Nouvelle École et après : Réussir ou mourir,0,tt38221275,1556160


Revisando en imdb estas peliculas salieron el 2025

In [27]:
dfnetflix4.loc[dfnetflix4['imdb_id'] == 'tt38203421', 'year'] = 2025
dfnetflix4.loc[dfnetflix4['imdb_id'] == 'tt38221275', 'year'] = 2025

#### Verificamos que el cambio de fecha se aplico correctamente

In [29]:
dfnetflix4[dfnetflix4['imdb_id'] == 'tt38203421']

,id,title,year,imdb_id,tmdb_id
2880,11039715,Rockstar: DUKI desde el fin del mundo,2025,tt38203421,1546933


In [30]:
dfnetflix4[dfnetflix4['imdb_id'] == 'tt38221275']

,id,title,year,imdb_id,tmdb_id
2890,11046064,Nouvelle École et après : Réussir ou mourir,2025,tt38221275,1556160


### Tipos de columnas

In [31]:
dfnetflix4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2908 entries, 0 to 2908
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       2908 non-null   int64 
 1   title    2908 non-null   object
 2   year     2908 non-null   int64 
 3   imdb_id  2908 non-null   object
 4   tmdb_id  2908 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 136.3+ KB


Los tipos de las columnas estan bien

### Analisis de columna `year`

In [33]:
dfnetflix4['year'].describe()

count    2908.000000
mean     2018.924347
std         7.290506
min      1967.000000
25%      2018.000000
50%      2021.000000
75%      2023.000000
max      2025.000000
Name: year, dtype: float64

## Creacion archivo `.csv`
Como ya tenemos los datos correctos y limpios, procedemos a guardar en un archivo `.csv`

In [34]:

dfnetflix4.to_csv('data/api_netflix.csv', index=False, encoding='utf-8')
